# Data Class

In [2]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()

    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #Score of 4 or 5
            return Sentiment.POSITIVE
        
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def evenly_distribute(self):
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)
        

# Load Data

In [3]:
import json

file_name = './books_small_10000.json'

reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text



'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

# Prep Data

In [4]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)

cont = ReviewContainer(training)

cont.evenly_distribute()

len(cont.reviews)

872

In [92]:
X_train = [x.text for x in training]
y_train = [x.sentiment for x in training]

X_test = [x.text for x in test]
y_test = [x.sentiment for x in test]

# Bags of Words Vectorization

In [93]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# fit and transform training data
X_train_vectors = vectorizer.fit_transform(X_train)

# transform test data
X_test_vectors = vectorizer.transform(X_test)



# Classification

### Linear SVM (Support Vector Machine)

In [94]:
from sklearn import svm

clf_svm = svm.SVC(kernel='linear')

clf_svm.fit(X_train_vectors, y_train)

print(X_test[0])
clf_svm.predict(X_test_vectors[0])

was sent an Arc of this book for an honest review and here it is = This is the kind of book that you want to read while sitting in front of the fire with a cup of hot apple cider and a blanket over your legs.I have read many of Jaci Burton's books and have never been disappointed. This first book in her new Hope series does not disappoint either.This is the story of Emma, a new vet who has come back home to open her own practice and Luke McCormack, a police officer in the same town.Both have been previously burned by love so both have issues but, that doesn't stop them from acting on that attraction.This book pulls you in from the first page, wraps you up and doesn't let you go until the end.I loved it!


array(['POSITIVE'], dtype='<U8')

### Decision Tree

In [95]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()

clf_dec.fit(X_train_vectors, y_train)

clf_dec.predict(X_test_vectors[0])

array(['POSITIVE'], dtype='<U8')

### Naive Bayes

In [96]:
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()

clf_gnb.fit(X_train_vectors.toarray(), y_train)

clf_gnb.predict(X_test_vectors[0].toarray())

array(['POSITIVE'], dtype='<U8')

### Logistic Regression

In [97]:
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression()

clf_log.fit(X_train_vectors, y_train)

clf_log.predict(X_test_vectors[0])

c:\Users\Matt\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array(['POSITIVE'], dtype='<U8')

## Evaluation

In [98]:
# MEAN ACCURACY
# Comparison of test vectors and test data
# .score() method returns the mean accuracy on the given test data and labels.
print(clf_svm.score(X_test_vectors, y_test))
print(clf_dec.score(X_test_vectors, y_test))
print(clf_gnb.score(X_test_vectors.toarray(), y_test))
print(clf_log.score(X_test_vectors, y_test))

0.8124242424242424
0.766969696969697
0.6587878787878788
0.8409090909090909


In [99]:
# F1 SCORE
# The F1 score can be interpreted as a weighted average of the precision and recall,
# where an F1 score reaches its best value at 1 and worst score at 0.
from sklearn.metrics import f1_score

f1_score(y_test, clf_svm.predict(X_test_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
# output: array([0.91319444, 0.21052632, 0.22222222])
# this result shows that the model is good at predicting positive reviews, but not good at predicting neutral and negative reviews


array([0.90738061, 0.2656    , 0.40268456])

In [104]:
# DEV ONLY
# Comparing the counts of each sentiment for model evaluation
y_train.count(Sentiment.NEGATIVE) + y_train.count(Sentiment.NEUTRAL)

1089